In [1]:
import os
import xarray as xr
import pandas as pd
from glob import glob
import geopandas as gpd
from datetime import datetime as dt
from datetime import timedelta as td

In [2]:
PATH_WEATHER        = os.path.join('..', '..', 'resources', 'reanalysis-era5-single-levels')
PATH_COMPLETE_FILES = glob(os.path.join(PATH_WEATHER, 'complete_files', '*.grib'))

In [3]:
def createNcFilesMonths(path_file) :

    ds = xr.open_dataset(path_file)
    name = os.path.splitext(os.path.basename(path_file))[0].split('_')[0]
    
    date_range      = pd.date_range(start=ds.time.values[0], end=ds.time.values[-1], freq='MS')
    list_year_month = [
        {
            'year': year, 
            'month': sorted(set(date_range[date_range.year == year].month))}
            for year in sorted(set(date_range.year))
    ]
    
    for year_month in list_year_month :
        for month in year_month['month'] :
            year = year_month['year']
            ds_filter = ds.sel(time=ds['time'].dt.year.isin([year]) & ds['time'].dt.month.isin([month]))
    
            path_out = os.path.join(PATH_WEATHER, name)
            if os.path.isdir(path_out) == False :
                os.makedirs(path_out)
                            
            path_out  = os.path.join(path_out, f'{name}_{year}-{month}.nc')
            ds_filter.to_netcdf(path_out)
        

In [4]:
if __name__ == '__main__' :
    for path_file in PATH_COMPLETE_FILES :
        createNcFilesMonths(path_file)